In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv('Cancer.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.drop(['Unnamed: 32','id'],inplace=True,axis=1)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# VOTING TECHNIQUE 

In [4]:
df['texture_mean']=np.sqrt(np.sqrt(np.sqrt(df['texture_mean'])))
df['perimeter_mean']=np.log(np.log(np.log(df['perimeter_mean'])))
df['smoothness_mean']=np.sqrt(np.sqrt(np.sqrt(df['smoothness_mean'])))
df['compactness_mean']=np.log(df['compactness_mean'])
df['symmetry_mean']=np.log(df['symmetry_mean'])
df['fractal_dimension_mean']=np.log(np.sqrt(df['fractal_dimension_mean']))
df['texture_se']=np.log(df['texture_se'])
df['smoothness_se']=np.log(np.sqrt(df['smoothness_se']))
df['area_se']=np.log(np.log(np.log(df['area_se'])))
df['concave points_se']=np.sqrt(df['concave points_se'])
df['symmetry_se']=np.log(np.sqrt(df['symmetry_se']))
df['fractal_dimension_se']=np.log(np.sqrt(np.sqrt(df['fractal_dimension_se'])))

In [5]:
f_drop=['radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst',\
        'concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst','radius_mean',\
        'concavity_se','perimeter_se','concavity_mean','area_mean','compactness_se','radius_se','concave points_mean']
df=df.drop(f_drop,axis=1)

In [6]:
from sklearn.preprocessing import LabelEncoder
lb_make=LabelEncoder()
df["diagnosis_code"]=lb_make.fit_transform(df["diagnosis"])
df[["diagnosis","diagnosis_code"]].head()

,diagnosis,diagnosis_code
0,M,1
1,M,1
2,M,1
3,M,1
4,M,1


In [7]:
df=df.drop('diagnosis',axis=1)
df.head()

,texture_mean,perimeter_mean,smoothness_mean,compactness_mean,symmetry_mean,fractal_dimension_mean,texture_se,area_se,smoothness_se,concave points_se,symmetry_se,fractal_dimension_se,diagnosis_code
0,1.339753,0.451593,0.765895,-1.281574,-1.419231,-1.270993,-0.099489,0.479970,-2.525807,0.125976,-1.752779,-1.271084,1
1,1.432884,0.461915,0.734533,-2.542875,-1.708154,-1.435255,-0.309382,0.378307,-2.627150,0.115758,-2.138293,-1.411473,1
2,1.465277,0.459061,0.758536,-1.833207,-1.575520,-1.406789,-0.239654,0.414572,-2.545652,0.143457,-1.897120,-1.347006,1
3,1.457641,0.385587,0.783839,-1.259133,-1.348228,-1.164259,0.144966,0.178339,-2.349191,0.136638,-1.409798,-1.171921,1
4,1.394982,0.464025,0.750175,-2.018911,-1.709811,-1.416552,-0.246796,0.415204,-2.233139,0.137295,-2.021066,-1.318894,1


# Logistic Regression Model

In [8]:
x=df.drop(['diagnosis_code','fractal_dimension_se','symmetry_se','texture_se'],axis=1)
y=df.diagnosis_code

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=4)

logmodel=LogisticRegression()
logmodel.fit(x_train,y_train)

predictions=logmodel.predict(x_test)

r=confusion_matrix(y_test,predictions)
print(r)

[[107  10]
 [  3  51]]


In [9]:
print("accuracy:",accuracy_score(y_test,predictions)*100)
print("True Positive Rate (TPR) :",(r[0][0]/(r[0][0]+r[1][0]))*100)
print("True Negative Rate (TNR) :",(r[1][1]/(r[1][1]+r[0][1]))*100)
print("False Negative Rate (FNR) :",(r[1][0]/(r[1][0]+r[0][0]))*100)
print("False Positive Rate (FPR) :",(r[0][1]/(r[0][1]+r[1][1]))*100)
x1=(classification_report(y_test,predictions))
print(x1)

accuracy: 92.39766081871345
True Positive Rate (TPR) : 97.27272727272728
True Negative Rate (TNR) : 83.60655737704919
False Negative Rate (FNR) : 2.727272727272727
False Positive Rate (FPR) : 16.39344262295082
              precision    recall  f1-score   support

           0       0.97      0.91      0.94       117
           1       0.84      0.94      0.89        54

   micro avg       0.92      0.92      0.92       171
   macro avg       0.90      0.93      0.91       171
weighted avg       0.93      0.92      0.93       171



# Decision Tree Model

In [10]:
x=df.drop(['diagnosis_code'],axis=1)
y=df.diagnosis_code

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=4)

classifier_entropy=DecisionTreeClassifier(criterion='entropy',random_state=42,max_depth=3)

classifier_entropy.fit(x_train,y_train)

predictions=classifier_entropy.predict(x_test)

r=confusion_matrix(y_test,predictions)
print(r)

[[105  12]
 [  7  47]]


In [11]:
print("accuracy:",accuracy_score(y_test,predictions)*100)
print("True Positive Rate (TPR) :",(r[0][0]/(r[0][0]+r[1][0]))*100)
print("True Negative Rate (TNR) :",(r[1][1]/(r[1][1]+r[0][1]))*100)
print("False Negative Rate (FNR) :",(r[1][0]/(r[1][0]+r[0][0]))*100)
print("False Positive Rate (FPR) :",(r[0][1]/(r[0][1]+r[1][1]))*100)
x1=(classification_report(y_test,predictions))
print(x1)

accuracy: 88.88888888888889
True Positive Rate (TPR) : 93.75
True Negative Rate (TNR) : 79.66101694915254
False Negative Rate (FNR) : 6.25
False Positive Rate (FPR) : 20.33898305084746
              precision    recall  f1-score   support

           0       0.94      0.90      0.92       117
           1       0.80      0.87      0.83        54

   micro avg       0.89      0.89      0.89       171
   macro avg       0.87      0.88      0.87       171
weighted avg       0.89      0.89      0.89       171



# KNN Model

In [12]:
x=df.drop(['diagnosis_code','fractal_dimension_se','symmetry_se','texture_se'],axis=1)
y=df.diagnosis_code

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=4,test_size=0.3)

sc_x=StandardScaler()
x_train=sc_x.fit_transform(x_train)
x_test=sc_x.fit_transform(x_test)

classifier=KNeighborsClassifier(n_neighbors=11,p=2,metric='euclidean')
classifier.fit(x_train,y_train)

predictions=classifier.predict(x_test)

r=confusion_matrix(y_test,predictions)
print(r)

[[110   7]
 [  0  54]]


In [13]:
print("accuracy:",accuracy_score(y_test,predictions)*100)
print("True Positive Rate (TPR) :",(r[0][0]/(r[0][0]+r[1][0]))*100)
print("True Negative Rate (TNR) :",(r[1][1]/(r[1][1]+r[0][1]))*100)
print("False Negative Rate (FNR) :",(r[1][0]/(r[1][0]+r[0][0]))*100)
print("False Positive Rate (FPR) :",(r[0][1]/(r[0][1]+r[1][1]))*100)
x1=(classification_report(y_test,predictions))
print(x1)

accuracy: 95.90643274853801
True Positive Rate (TPR) : 100.0
True Negative Rate (TNR) : 88.52459016393442
False Negative Rate (FNR) : 0.0
False Positive Rate (FPR) : 11.475409836065573
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       117
           1       0.89      1.00      0.94        54

   micro avg       0.96      0.96      0.96       171
   macro avg       0.94      0.97      0.95       171
weighted avg       0.96      0.96      0.96       171



# Naive Baye's Model

In [14]:
x=df.drop(['diagnosis_code','fractal_dimension_se','texture_se',],axis=1)
y=df.diagnosis_code

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=4,test_size=0.3)

model=GaussianNB()
model.fit(x_train,y_train)

predictions = model.predict(x_test)

r=confusion_matrix(y_test,predictions)
print(r)

[[106  11]
 [  2  52]]


In [15]:
print("accuracy:",accuracy_score(y_test,predictions)*100)
print("True Positive Rate (TPR) :",(r[0][0]/(r[0][0]+r[1][0]))*100)
print("True Negative Rate (TNR) :",(r[1][1]/(r[1][1]+r[0][1]))*100)
print("False Negative Rate (FNR) :",(r[1][0]/(r[1][0]+r[0][0]))*100)
print("False Positive Rate (FPR) :",(r[0][1]/(r[0][1]+r[1][1]))*100)
x1=(classification_report(y_test,predictions))
print(x1)

accuracy: 92.39766081871345
True Positive Rate (TPR) : 98.14814814814815
True Negative Rate (TNR) : 82.53968253968253
False Negative Rate (FNR) : 1.8518518518518516
False Positive Rate (FPR) : 17.46031746031746
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       117
           1       0.83      0.96      0.89        54

   micro avg       0.92      0.92      0.92       171
   macro avg       0.90      0.93      0.92       171
weighted avg       0.93      0.92      0.93       171



# Bagging Ensemble Model

In [16]:
X=df.drop("diagnosis_code",axis=1)
Y=df["diagnosis_code"]

seed=7
num_trees=100
max_features=3
kfold=model_selection.KFold(n_splits=10,random_state=seed)
model=rfc(n_estimators=num_trees,max_features=max_features)
results=model_selection.cross_val_score(model,X,Y,cv=kfold)
print(results.mean()*100)

93.14223057644112


# Voting Ensemble Model

In [17]:
X=df.drop("diagnosis_code",axis=1)
Y=df["diagnosis_code"]

seed=7
kfold=model_selection.KFold(n_splits=10,random_state=seed)
estimators=[]
model1=LogisticRegression()
estimators.append(('logistic',model1))
model2=DecisionTreeClassifier()
estimators.append(('dect',model2))
model3=KNeighborsClassifier()
estimators.append(('knn',model3))
model4=GaussianNB()
estimators.append(('nb',model4))

In [18]:
ensemble=VotingClassifier(estimators)
results=model_selection.cross_val_score(ensemble,X,Y,cv=kfold)
print(results.mean()*100)

90.86466165413535
